Notebook for harry testing

In [25]:
import requests
import sqlite3
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt


db_path = r"C:\Users\Harry Hall\OneDrive - UNSW\Desktop\MASTERS\S1\Research & Innovation\laliga.sqlite"
conn = sqlite3.connect(db_path)
df = pd.read_sql_query("SELECT * FROM Matches", conn)
conn.close()
display(df.head())

conn.close()

,season,division,matchday,date,time,home_team,away_team,score
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2


Now need to clean the data and process into usable features.  
  
Firslty, dropping rows with no score and then splitting score into home and away component.  
  
I think it also makes sense to use the final notation suggested in the assignment of: home_win = 1, away_win = 2, tie = X  
  

In [26]:
df = df.dropna(axis = 'index' , how = 'any', subset = 'score')
df[['home_score', 'away_score']] = df['score'].str.split(':', expand=True)
df['home_score'] = pd.to_numeric(df['home_score'])
df['away_score'] = pd.to_numeric(df['away_score'])
df = df[(~df['home_score'].isnull()) | (~df['away_score'].isnull())]

df['result'] = df.apply(lambda row: '1' if row['home_score'] > row['away_score']
                                        else '2' if row['home_score'] < row['away_score']
                                        else 'X', axis=1)



-goals scores (at home and away)  
-goals conceded (at home and away)  
-goal difference  
-current ranking  
-average ranking in the last 5 seasons for example  
-current streak of games (average number of points gained in the last X games)  
-head2head (goals scored, conceded, wins, ties...)  
-general win/tie/loses rate of each team  
-Number of promotions and relegations ?  
-biggest win ?  
-longest undefeated streak ?  
-day and hour ?  

In [ ]:
df['gd_5_game_home'] = None
df['gd_5_game_away'] = None
df['form_10_game_home'] = None
df['form_10_game_away'] = None

df['live_rank_home'] = None
df['live_rank_away'] = None

def calculate_gd_5_game(df):
    teams = df['home_team'].unique()
    for team in teams:
        team_df = df[(df['home_team'] == team) | (df['away_team'] == team)].copy()
        team_df['goal_difference'] = team_df.apply(
            lambda x: x['home_score'] - x['away_score'] if x['home_team'] == team else x['away_score'] - x['home_score'], axis=1
        )
        team_df['gd_5_game'] = team_df['goal_difference'].rolling(window=5).sum().shift(1)
        
        # Assign gd_5_game to the appropriate column based on home or away
        df.loc[team_df[team_df['home_team'] == team].index, 'gd_5_game_home'] = team_df.loc[team_df['home_team'] == team, 'gd_5_game']
        df.loc[team_df[team_df['away_team'] == team].index, 'gd_5_game_away'] = team_df.loc[team_df['away_team'] == team, 'gd_5_game']
    
    return df

def calculate_form_10_game(df):
    teams = df['home_team'].unique()
    for team in teams:
        team_df = df[(df['home_team'] == team) | (df['away_team'] == team)].copy()
        team_df['win'] = team_df.apply(
            lambda x: 1 if (x['home_team'] == team and x['home_score'] > x['away_score']) or 
                           (x['away_team'] == team and x['away_score'] > x['home_score']) else 0, axis=1
        )
        team_df['form_10_game'] = team_df['win'].rolling(window=10).sum().shift(1)
        
        # Assign form_10_game to the appropriate column based on home or away
        df.loc[team_df[team_df['home_team'] == team].index, 'form_10_game_home'] = team_df.loc[team_df['home_team'] == team, 'form_10_game']
        df.loc[team_df[team_df['away_team'] == team].index, 'form_10_game_away'] = team_df.loc[team_df['away_team'] == team, 'form_10_game']
    
    return df

# THIS FUNCTION NEEDS WORK
def calculate_live_rank(df):
    teams = df['home_team'].unique()
    points_table = {team: 0 for team in teams}
    matchdays = sorted(df['matchday'].unique())
    
    for matchday in matchdays:
        day_df = df[df['matchday'] == matchday]
        for _, row in day_df.iterrows():
            home_team, away_team = row['home_team'], row['away_team']
            home_score, away_score = row['home_score'], row['away_score']
            
            # Update points based on match result
            if home_score > away_score:
                points_table[home_team] += 3
            elif away_score > home_score:
                points_table[away_team] += 3
            else:
                points_table[home_team] += 1
                points_table[away_team] += 1
        
        # Sort teams by points and assign ranks
        sorted_teams = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
        rank_dict = {team: rank+1 for rank, (team, _) in enumerate(sorted_teams)}
        
        # Update live rank for this matchday for both home and away teams
        for _, row in day_df.iterrows():
            df.loc[row.name, 'live_rank_home'] = rank_dict[row['home_team']]
            df.loc[row.name, 'live_rank_away'] = rank_dict[row['away_team']]
    
    return df


df = calculate_gd_5_game(df)
df = calculate_form_10_game(df)
df = calculate_live_rank(df)


display(df.head())
display(df.tail())


,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result,gd_5_game_home,gd_5_game_away,form_10_game_home,form_10_game_away,live_rank_home,live_rank_away
0,1928-1929,1,1,2/10/29,None,Arenas Club,Athletic Madrid,2:3,2,3,2,NaN,NaN,NaN,NaN,78,67
1,1928-1929,1,1,2/10/29,None,Espanyol,Real Unión,3:2,3,2,1,NaN,NaN,NaN,NaN,7,59
2,1928-1929,1,1,2/10/29,None,Real Madrid,Catalunya,5:0,5,0,1,NaN,NaN,NaN,NaN,2,133
3,1928-1929,1,1,2/10/29,None,Donostia,Athletic,1:1,1,1,X,NaN,NaN,NaN,NaN,99,6
4,1928-1929,1,1,2/12/29,None,Racing,Barcelona,0:2,0,2,2,NaN,NaN,NaN,NaN,10,1


,season,division,matchday,date,time,home_team,away_team,score,home_score,away_score,result,gd_5_game_home,gd_5_game_away,form_10_game_home,form_10_game_away,live_rank_home,live_rank_away
48345,2021-2022,2,3,8/28/21,10:00 PM,Ponferradina,Girona,2:1,2,1,1,-3.0,4.0,4.0,8.0,60,44
48346,2021-2022,2,3,8/29/21,5:00 PM,SD Amorebieta,UD Almería,2:1,2,1,1,NaN,5.0,NaN,5.0,141,39
48347,2021-2022,2,3,8/29/21,7:30 PM,CD Lugo,Real Valladolid,0:2,0,2,2,2.0,7.0,3.0,6.0,66,14
48348,2021-2022,2,3,8/29/21,7:30 PM,Real Sociedad B,CF Fuenlabrada,0:0,0,0,X,NaN,1.0,NaN,4.0,126,86
48349,2021-2022,2,3,8/29/21,10:00 PM,Burgos CF,SD Eibar,0:1,0,1,2,-4.0,-1.0,1.0,4.0,45,30


In [28]:
df['prev_season_rank_home'] = 'Unranked'
df['prev_season_rank_away'] = 'Unranked'

def calculate_season_ranks(df):

    prev_season_ranks = {}

    seasons = df['season'].unique()
    for season in seasons:

        season_df = df[df['season'] == season].copy()

        points_table = {}

        for _, row in season_df.iterrows():
            home_team, away_team = row['home_team'], row['away_team']
            home_score, away_score = row['home_score'], row['away_score']

            if home_team not in points_table:
                points_table[home_team] = 0
            if away_team not in points_table:
                points_table[away_team] = 0

            if home_score > away_score:
                points_table[home_team] += 3
            elif away_score > home_score:
                points_table[away_team] += 3
            else:
                points_table[home_team] += 1
                points_table[away_team] += 1

        ranked_teams = sorted(points_table.items(), key=lambda x: x[1], reverse=True)
        season_ranking = {team: rank+1 for rank, (team, _) in enumerate(ranked_teams)}

        for team, rank in season_ranking.items():
            prev_season_ranks[(season, team)] = rank
    
    return prev_season_ranks

prev_season_ranks = calculate_season_ranks(df)


def assign_prev_season_rank(df, prev_season_ranks):

    for i, row in df.iterrows():
        current_season = row['season']
        home_team = row['home_team']
        away_team = row['away_team']

        prev_season_year = str(int(current_season.split('-')[0]) - 1) + '-' + str(int(current_season.split('-')[1]) - 1)

        df.at[i, 'prev_season_rank_home'] = prev_season_ranks.get((prev_season_year, home_team), None)
        df.at[i, 'prev_season_rank_away'] = prev_season_ranks.get((prev_season_year, away_team), None)
    
    return df

df = assign_prev_season_rank(df, prev_season_ranks)

display(df[['season', 'home_team', 'away_team', 'prev_season_rank_home', 'prev_season_rank_away']].head())
display(df[['season', 'home_team', 'away_team', 'prev_season_rank_home', 'prev_season_rank_away']].tail())


,season,home_team,away_team,prev_season_rank_home,prev_season_rank_away
0,1928-1929,Arenas Club,Athletic Madrid,None,None
1,1928-1929,Espanyol,Real Unión,None,None
2,1928-1929,Real Madrid,Catalunya,None,None
3,1928-1929,Donostia,Athletic,None,None
4,1928-1929,Racing,Barcelona,None,None


,season,home_team,away_team,prev_season_rank_home,prev_season_rank_away
48345,2021-2022,Ponferradina,Girona,15,9
48346,2021-2022,SD Amorebieta,UD Almería,None,8
48347,2021-2022,CD Lugo,Real Valladolid,26,41
48348,2021-2022,Real Sociedad B,CF Fuenlabrada,None,17
48349,2021-2022,Burgos CF,SD Eibar,None,42
